# Generics

## Trait Bound Syntax

Rust allows type definitions to have type parameters. In fact we already have seen some of these. The `Option<T>` and `Result<T,E>` types are good examples. These two types do not place constraints on their parameters but it is possible to do so.

In [ ]:
trait Lifeform {
    fn is_alive(&self) -> bool;
}

struct Couple<T: Lifeform> {
    male: T,
    female: T,
}

Trait bounds can be placed on functions :

In [ ]:
trait Lifeform {
    fn is_alive(&self) -> bool;
}

fn remove_dead_classic<T: Lifeform>(things: &mut Vec<T>) {
    things.retain(|thing| thing.is_alive())
}

fn remove_dead_impl(things: &mut Vec<impl Lifeform>) {
//                                   ^^^^^^^^^^^^^
//                         syntactic sugar for generics
    things.retain(|thing| thing.is_alive())
}

Multiple trait bounds can be specified with the *+* syntax

In [ ]:
trait Lifeform {
    fn is_alive(&self) -> bool;
}

fn remove_dead<T: Lifeform + Copy>(things: &mut Vec<T>) {
    things.retain(|thing| thing.is_alive())
}

With generic programming, Rust provide static dispatch with mono-morphisation:

In [ ]:
trait Lifeform {
    fn is_alive(&self) -> bool;
}

#[derive(Debug)]
struct Kelp { size: i32 }

impl Lifeform for Kelp {
    fn is_alive(&self) -> bool { self.size <= 0 }
}

#[derive(Debug)]
struct Fish { age: usize }

impl Lifeform for Fish {
    fn is_alive(&self) -> bool { self.age >= 20 }
}

fn remove_dead<T: Lifeform>(things: &mut Vec<T>) {
    things.retain(|thing| thing.is_alive())
}

fn main() {
    let mut kelps = vec![Kelp{size: 10}, Kelp{size: 0}];
    let mut fishes = vec![Fish{age: 21}, Fish{age: 3}];
    
    remove_dead(&mut kelps);
    //          ^^^^^^^^^^ &mut Vec<Kelp>
    remove_dead(&mut fishes);
    //          ^^^^^^^^^^^ &mut Vec<Fish>
    
    println!("kelps = {kelps:?}");
    println!("fishes = {fishes:?}");
}
main();

## Dynamic dispatch (trait objects)

Rust also support dynamic dispatch using the `dyn Trait` syntax.

In [ ]:
trait Lifeform {
    fn is_alive(&self) -> bool;
    fn eat(&mut self);
}

#[derive(Debug)]
struct Kelp { size: i32 }

impl Lifeform for Kelp {
    fn is_alive(&self) -> bool { self.size <= 0 }
    fn eat(&mut self) { self.size += 1; }
}

#[derive(Debug)]
struct Fish { age: usize }

impl Lifeform for Fish {
    fn is_alive(&self) -> bool { self.age >= 20 }
    fn eat(&mut self) { self.age += 1; }
}

fn simulate(thing: &mut dyn Lifeform) {
    if thing.is_alive() {
        thing.eat();
    }
}

fn main() {
    let mut kelp = Kelp{size: 10};
    let mut fish = Fish{age: 21};
    
    simulate(&mut kelp);
    //       ^^^^^^^^^ &mut Kelp
    simulate(&mut fish);
    //       ^^^^^^^^^ &mut Fish
    
    println!("{kelp:?}");
    println!("{fish:?}");
    println!("sizeof &mut Kelp = {}", std::mem::size_of::<&mut Kelp>());
    println!("sizeof &mut dyn Lifeform = {}", std::mem::size_of::<&mut dyn Lifeform>());
}
main();

## `impl` in return position

We can also use the `impl Trait` syntax in the return position to return a value of some type that implements a trait, as shown here:

In [ ]:
trait Lifeform {
    fn is_alive(&self) -> bool;
}

#[derive(Debug)]
struct Kelp { size: i32 }

impl Lifeform for Kelp {
    fn is_alive(&self) -> bool { self.size <= 0 }
}

fn return_lifeform() -> impl Lifeform { // <-- here
    Kelp { size: 10 }
}

By using `impl Lifeform` for the return type, we specify that the `return_lifeform` function returns some type that implements the `Lifeform` trait without naming the concrete type. 

The ability to return a type that is only specified by the trait it implements is especially useful in the context of closures and iterators. Closures and iterators create types that only the compiler knows or types that are very long to specify.  

However, you can only use `impl Trait` if you’re returning a single type. For example, in the previous code, you could not return either a `Kelp` or a `Fish`.

In [ ]:
fn adder(value: i32) -> impl Fn(i32) -> i32 {
    move |x| x + value
}
adder(5)(3)

When the trait bounds become complex, they can be put in a `where` clause in the function's signature :

In [ ]:
use std::fmt::Display;
use std::fmt::Debug;

fn some_function<T, U>(t: T, u: U) -> i32
    where T: Display + Clone,  // <-- where clause
          U: Clone + Debug
{ 0 }

Using trait bounds to conditionally implement methods
----

By using a trait bound with an `impl` block that uses generic type parameters, we can implement methods conditionally for types that implement the specified traits.

In [ ]:
trait Lifeform {
    fn is_alive(&self) -> bool;
}

struct Couple<T> {
    male: T,
    female: T,
}

impl<T> Couple<T> {
    fn new(male: T, female: T) -> Self {
        Self {male, female}
    }
}

impl<T: Lifeform + Default> Couple<T> {
    fn mate(&self) -> Option<T> {
        if self.male.is_alive() && self.female.is_alive() {
            Some(T::default())
        } else {
            None
        }
    }
}

Exercises
===

**Exercice 1:** Simplify the code below to use a generic approach

In [ ]:
fn largest_i32(list: &[i32]) -> i32 {  // find the largest i32 in a slice
    let mut largest = list[0];

    for &item in list.iter() {
        if item > largest {
            largest = item;
        }
    }

    largest
}

fn largest_char(list: &[char]) -> char { // find the largest char in a slice
    let mut largest = list[0];

    for &item in list.iter() {
        if item > largest {
            largest = item;
        }
    }

    largest
}

fn main() {
    let number_list = vec![34, 50, 25, 100, 65];

    let result = largest_i32(&number_list);
    println!("The largest number is {}", result);

    let char_list = vec!['y', 'm', 'a', 'q'];

    let result = largest_char(&char_list);
    println!("The largest char is {}", result);
}
main();

> **Tip:** : Use the trait [`std::cmp::Ord`](https://doc.rust-lang.org/std/cmp/trait.Ord.html) and [`std::marker::Copy`](https://doc.rust-lang.org/std/marker/trait.Copy.html).

**Solution :**

In [ ]:
fn largest<T: PartialOrd + Copy>(list: &[T]) -> T {
    let mut largest = list[0];

    for &item in list.iter() {
        if item > largest {
            largest = item;
        }
    }

    largest
}

fn main() {
    let number_list = vec![34, 50, 25, 100, 65];

    let result = largest_idiomatic(&number_list);
    println!("The largest number is {}", result);

    let char_list = vec!['y', 'm', 'a', 'q'];

    let result = largest_idiomatic(&char_list);
    println!("The largest char is {}", result);
}
main();